## Set Up / Imports

Main items of importance here are GPT-2, markovify, and a few other python libraries for later use.

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install markovify
!pip install dominate

In [ ]:
# GPT-2 Stuff

import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

# Added Stuff

import random
import markovify
import dominate
from dominate.tags import *

In [ ]:
# Class(es)
# Don't necessarily need these, but here's to practice

class character(object):
    def __init__(self,name,age,race,title,profile,opinions):
        self.name = name
        self.age = age
        self.race = race
        self.title = title
        self.profile = profile
        self.opinions = opinions

## Creating Initial Character Profile

Build initial cahracter profile off of some more basic generation. I.E. Random selection.

In [ ]:
# Generate name and title
def generateName():
    with open("btn_givennames.txt",encoding="UTF-8") as namebank:
        text = namebank.read()

        # remove /ignore gender markings
        # and pick a name at random
        text = text.replace("	m","").replace("	f","")
        names = text.split()
        name = random.choice(names)

        return name

# Generate an age from 25 - 100
# because fiction is too obsessed with teenagers
def generateAge():
    age = random.randint(25,100)
    return age

# Give the character a title like The Stone-Crusher or something
def generateTitle():
  colors = ["red","blue","green","yellow","pink","purple",
            "orange","black","white","brown","grey","silver","gold"]
  jobs = ["necromancer","paladin","priest","warlock","ranger","bard"
             "thief","scout","cleric","templar","gunslinger","alchemist"]
  things = ["stone","grass","water","mist","music","life",
            "blood", "land","kingdom","mountain",]
  descript = ["crusher","cutter","waster","healer","saviour","fool",
              "taker","enigma"]

  color = random.choice(colors).title()
  job = random.choice(jobs).title()
  thing = random.choice(things).title()
  doer = random.choice(descript).title()

# Less chance of thing-ish title because they're less options
  randint = random.randint(0,2)
  if randint == 0:
    title = "the "+color+" "+job+""
  if randint == 1:
    title = "the "+color+" "+job+""
  elif randint == 2:
    title = "the "+thing+" "+doer+""
  return title

def generateRace():
  races = ["human","elf","dwarf","orc","goblin",
           "beast man", "fae", "gnome", "lizard",
           "daemon", "elemental"]
           
  race = random.choice(races)
  return race

# Have to call this now to use with gpt-2 prefix
name = generateName()
age = generateAge()
title = generateTitle()
race = generateRace()

chara = character(name,age,race,title,"","")

## Generating More of Profile

Taking the information created above, use GPT-2 to generate a custom profile and short profile for the character. While I wanted to use a fine-tuned model, I ran into too many issues with RAM and having to re-boot page to use it effectively for the assignment.

Wordcount is here too much of a wildcard to quantify.

In [ ]:
# Generate About using GPT-2 and character name
def generateProfile():
    profile = gpt2.generate(sess,
              model_name=model_name,
              prefix="Name: "+chara.name+"\nRace: "+chara.race.title()+
                     "\nClass: "+chara.title.title()+"\nAge: "+str(chara.age)+"\nHobbies:",
              length=50,
              temperature=0.7,
              top_p=0.9,
              nsamples=1,
              batch_size=1
    )
    return profile

# Taking elements from prior cell,
# compile profile with GPT-2 generation

chara.profile = str(generateProfile())

## Conducting an Interview

Again, using some of the previous information, conduct an interview to try and pin down more about the character.

Try to force a format by using particular prefixes. 50-80 words for each question.

In [ ]:
# 5 seems to be the limit before google colab crashes from RAM usage, haha
questions = [
  "Q: When asked about what kind of pets they prefer, "+chara.name+" said\n\nA:",
  "Q: When asked what their favorite color is, "+chara.title+" said\n\nA:",
  "Q: When asked what age they would like to live to, "+chara.name+", "+str(chara.age)+", said\n\nA:",
  "Q: When asked what’s the farthest they've ever been from home, "+chara.name+", said\n\nA:",
  "Q: When asked what they consider to be their best magical find, "+chara.name+" said\n\nA:"
]
opinions = ""
counter = 0

def generateOpinion(counter):
    count = counter
    opinion = gpt2.generate(sess,
              model_name=model_name,
              prefix=questions[count],
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=1,
              batch_size=1
    )
    return opinion

while counter < 2:
#while counter < len(questions):
  opinions += str(generateOpinion(counter))
  opinions += "--\n\n"
  counter += 1

chara.opinions = opinions

## Building a Story

Now, taking the character profile, and taking several edited sources of text with specific marked variables (like [NAME]), create and mix a story using markovify.

In the future I'd let the user choose genres based on number input, but for now, we're stuck with aesop's fables because they're the easiest to edit, name-wise.

In [58]:
# Get texts for Markovify

with open("aesop.txt",encoding="UTF-8") as aesop:
    text = aesop.read()
with open("tales.txt",encoding="UTF-8") as tales:
    text2 = tales.read()

textA = markovify.Text(text)
textB = markovify.Text(text2)

novel = ""

for i in range(5):
  novel += str(textA.make_sentence())
  novel += "\n"

print(novel)

For a long time he jumped he missed it by a long time ago believed that he was overcome with grief and despair.
The Man in the stars.
None
His eyes were fixed on the hurt and kiss it.
When the poor Fox Without a Tail got up and made a long way.



## Exporting to Document

Take all created outputs, etc, and combine into a document for easy saving / sharing. Ta-da! You've created a character with the help of a computer.

In [ ]:
# something with dominate

# SOURCES

## Namebank:
- Local: btn_givennames.txt
- From: https://www.behindthename.com/api/

## Text
- Aesop's Fables
  - Local: aesop.txt
  - From: http://read.gov/aesop/001.html
  - Note: Grabbed fables that had fewer characters, so that names/titles could be edited more easily
-Irish Fairy Tales by James Stephens
 - Local: tales.txt
 - From: https://www.gutenberg.org/files/2892/2892-h/2892-h.htm


# GPT-2 LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.